In [2]:
from __future__ import annotations

from etils.lazy_imports import *
import notion_client

import common

In [68]:
common.add_notion_to_sys_path()

PosixPath('/Users/epot/__A__/code/automations/applications')

In [93]:
ecolab.clear_cached_modules(['auto_notion'])

import auto_notion

In [94]:
db = auto_notion.Database('2f1baf0d3c114887ab7a32912a235589')
db

In [95]:
all_rows = list(db)
all_rows = sorted(all_rows, key=lambda r: r.created_time)

In [96]:
def to_list(values: str | None) -> list[int]:
    if values is None:
        return []
    elif not values.strip():
        return []
    results = []
    for val in values.split(','):
        if val.endswith('*'):
            val = val.rstrip('*')
            val = int(val) / 2
        else:
            val = int(val)
        results.append(val)
    return results

In [169]:
ecolab.clear_cached_modules(['plotly_dc'])

import plotly_dc


@dataclasses.dataclass
class Session:
    sets: list[float]
    date: datetime.datetime

    @property
    def max(self) -> float:
        return max(self.sets, default=0)

    @property
    def total(self) -> float:
        return sum(self.sets)


@dataclasses.dataclass
class Excercise(plotly_dc.Visualizable):
    sessions: list[Session]

    @property
    def dates(self):
        return [sess.date for sess in self.sessions if sess.total]

    @property
    def max(self):
        return max(self.maxs)

    @property
    def max_total(self):
        return max(self.maxs)

    @property
    def maxs(self):
        return [sess.max for sess in self.sessions if sess.total]

    @property
    def totals(self):
        return [sess.total for sess in self.sessions if sess.total]

    @property
    def zeros(self):
        return [0 for sess in self.sessions if sess.total]
    
    @property
    def cum_max(self):
        curr = 0
        cums = []
        for v in self.maxs:
            if v > curr:
                curr = v
            cums.append(curr)
        return cums
    
    @property
    def cum_total(self):
        curr = 0
        cums = []
        for v in self.totals:
            if v > curr:
                curr = v
            cums.append(curr)
        return cums
    
    # TODO: Weekly agregate
    @property
    def week_df(self):
        # Warning: Mean can contain 0 values
        return self.df.groupby(pd.Grouper(key='dates', freq='W-MON')).agg(
            maxs_of_the_week=pd.NamedAgg(column="maxs", aggfunc="max"),
            weekly_volume=pd.NamedAgg(column="totals", aggfunc="sum"),
            zeros=pd.NamedAgg(column="zeros", aggfunc="sum"),
        )

    Y_SESSION = [
        'maxs',
        'totals',
        # 'cum_total',
        # 'cum_max',
        'zeros',
    ]
    _df_fields = [
        'dates',
    ] + Y_SESSION


def get_metric(excercise_name):
    return Excercise([Session(
        sets=to_list(getattr(row, excercise_name)),
        date=row.created_time,
    ) for row in all_rows])


pull_up = get_metric('pull_up')
push_up = get_metric('push_up')

In [ ]:
import plotly.express as px

In [165]:
px.line(push_up.week_df, y=['maxs_of_the_week', 'weekly_volume', 'zeros'])

In [166]:
px.line(pull_up.week_df, y=['maxs_of_the_week', 'weekly_volume', 'zeros'])

In [171]:
px.line(push_up.df, x='dates', y=pull_up.Y_SESSION)

In [168]:
for row in all_rows:
    vals = to_list(row.pull_up)
    if vals and not row.pull_total:
        row.pull_total = sum(vals)

    vals = to_list(row.push_up)
    if vals and not row.push_total:
        row.push_total = sum(vals)